In [1]:
import pandas as pd
import numpy as np


pdr = pd.read_csv('ratings.dat', delimiter='::', header=None, engine='python')
print(pdr)
print('Number of ratings:', len(pdr))
#print('Number of users:', len(pdr['uid'].unique()))
#print('Number of movies:', len(pdr['iid'].unique()))

            0     1  2          3
0           1  1193  5  978300760
1           1   661  3  978302109
2           1   914  3  978301968
3           1  3408  4  978300275
4           1  2355  5  978824291
...       ...   ... ..        ...
1000204  6040  1091  1  956716541
1000205  6040  1094  5  956704887
1000206  6040   562  5  956704746
1000207  6040  1096  4  956715648
1000208  6040  1097  4  956715569

[1000209 rows x 4 columns]
Number of ratings: 1000209


In [2]:
pdr.columns = ['uid', 'iid', 'rating', 'timestamp']
pdr

,uid,iid,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [3]:
print('user:', min(pdr.groupby('uid').agg('count').iid))
print('item:', min(pdr.groupby('iid').agg('count').uid))

user: 20
item: 1


In [4]:
new_pdr = pdr

In [5]:
from tqdm import tqdm

low = 5
while True:
    n_filter = 0
    # drop item
    print('>>> item:')
    user_data = new_pdr.groupby('iid').agg(list)
    for i in tqdm(new_pdr.iid.unique()):
        if len(user_data['uid'][i]) < low:
            new_pdr = new_pdr[new_pdr.iid.values != i]
            n_filter += 1

    # drop user
    print('>>> user:')
    item_data = new_pdr.groupby('uid').agg(list)
    for i in tqdm(new_pdr.uid.unique()):
        if len(item_data['iid'][i]) < low:
            new_pdr = new_pdr[new_pdr.uid.values != i]
            n_filter += 1
    
    # terminate
    if n_filter == 0:
        break

# reset index
new_pdr.reset_index(drop=True, inplace=True)

>>> item:


100%|██████████| 3706/3706 [00:03<00:00, 978.07it/s]  


>>> user:


100%|██████████| 6040/6040 [00:00<00:00, 236804.63it/s]


>>> item:


100%|██████████| 3416/3416 [00:00<00:00, 220807.28it/s]


>>> user:


100%|██████████| 6040/6040 [00:00<00:00, 241100.13it/s]


In [6]:
def is_squ(pdr):
    users, items = pdr['uid'].unique(), pdr['iid'].unique()
    n_user, n_item = len(users), len(items)
    if min(users) == 0 and max(users) == n_user-1 and min(items) == 0 and max(items) == n_item-1:
        return True
    else:
        return False

is_squ(new_pdr)

False

In [7]:
print(f'user id: {max(new_pdr.uid.unique())}/{len(new_pdr.uid.unique())}')
print(f'item id: {max(new_pdr.iid.unique())}/{len(new_pdr.iid.unique())}')

user id: 6040/6040
item id: 3952/3416


In [8]:
import numpy as np

# squeeze uid
new_pdr.uid -= 1
user_dict = {}
for i, ori in enumerate(new_pdr['uid'].unique()):
    user_dict[ori] = i

# squeeze iid
item_dict = {}
for i, ori in enumerate(new_pdr['iid'].unique()):
    item_dict[ori] = i

np.save('user_dict', user_dict)    
np.save('item_dict', item_dict)

new_pdr.uid = new_pdr.uid.map(user_dict)
new_pdr.iid = new_pdr.iid.map(item_dict)

# save file
new_pdr.sort_values(by=['uid', 'iid']).to_csv('ratings.csv', index=False)

In [9]:
is_squ(new_pdr)
print(len(new_pdr['uid'].unique()))
print(len(new_pdr['iid'].unique()))

6040
3416


train:test = 9:1

In [10]:
from tqdm import tqdm
import time 
import pandas
import random

train_ratio = 0.9
train_df = pd.DataFrame([], columns=['uid', 'iid', 'rating'])
test_df = pd.DataFrame([], columns=['uid', 'iid', 'rating'])

train_index = []
test_index = []
print(len(new_pdr.uid.unique()))
for i in range(len(new_pdr.uid.unique())):
    t1 = time.time()
    indices = new_pdr[new_pdr.uid == i].index
    index_list = indices.to_list()
    
    total = len(indices)

    n_train = int(total * train_ratio)
    
    train_idx = random.sample(index_list, n_train)
    test_idx = list(set(index_list) - set(train_idx))
    
    train_index.extend(train_idx)
    test_index.extend(test_idx)
    if i%1000 == 0:
        print(i)

train_index = np.array(train_index)
test_index = np.array(test_index)

user_train = new_pdr.iloc[np.sort(train_index), :]
user_test = new_pdr.iloc[np.sort(test_index), :]
    
train_df = train_df.append(user_train, ignore_index=True)
test_df = test_df.append(user_test, ignore_index=True)
    
train_df['val'] = np.float16(train_df['rating'])
test_df['val'] = np.float16(test_df['rating'])

train_df[['uid', 'iid', 'val']].to_csv('squ0_train.csv', header=None, index=False)
test_df[['uid', 'iid', 'val']].to_csv('squ0_test.csv', header=None, index=False)

6040
0
1000
2000
3000
4000
5000
6000


/tmp/ipykernel_2793704/196034843.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(user_train, ignore_index=True)
/tmp/ipykernel_2793704/196034843.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df = test_df.append(user_test, ignore_index=True)


In [12]:
train = pd.read_csv('squ0_train.csv', header=None, names=['uid', 'iid', 'rating'])
print(train)

         uid  iid  rating
0          0    0     5.0
1          0    1     3.0
2          0    2     3.0
3          0    3     4.0
4          0    4     5.0
...      ...  ...     ...
896909  6039  147     3.0
896910  6039  772     1.0
896911  6039  365     5.0
896912  6039  152     4.0
896913  6039   26     4.0

[896914 rows x 3 columns]


In [15]:
import pandas as pd
import numpy as np

train = pd.read_csv('squ0_test.csv', header=None, names=['uid', 'iid', 'rating'])

user_train = np.load('/home/zyz/paper_codes/lightRE/user_data_kmeans.npy', allow_pickle=True)

selected_rows_1 = train[train['uid'].isin(user_train)]

print(selected_rows_1)
selected_rows_1.to_csv('squ0_test_kmeans.csv', header=None, index=False)


           uid   iid  rating
571         43   327     1.0
572         43   216     4.0
573         43   226     4.0
574         43    44     5.0
575         43   612     5.0
...        ...   ...     ...
1031490  69870    14     4.0
1031491  69870   140     5.0
1031492  69870  1434     5.0
1031493  69870   368     2.0
1031494  69870  2564     4.0

[37122 rows x 3 columns]


2064
6206
